In [1]:
print("Hello World")

Hello World


# **Story Writing & Use case**
- I am going to analyze public GitHub repositories for Python projects to build a clean dataset for analytics.
I'll fetch repositories via GitHub API using authentication (Token), process and clean the data, apply filters, handle API failures, and store clean results into CSV.

In [11]:
import requests
import pandas as pd
from datetime import datetime

In [ ]:
GITHUB_TOKEN = "YOUR_GITHUB_TOKEN_HERE"

HEADERS = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}

In [ ]:
url = "https://api.github.com/search/repositories"
params = {
    "q": "language:python",
    "sort": "stars",
    "order": "desc",
    "per_page": 50
}
try:
    response = requests.get(url, headers=HEADERS, params=params, timeout=10)
    response.raise_for_status()
    data = response.json()
    if data:
        print("Data fetched successfully")

except requests.exceptions.Timeout:
    raise Exception("❌ Request timed out")

except requests.exceptions.HTTPError as e:
    raise Exception(f"❌ HTTP error occurred: {e}")

except requests.exceptions.RequestException as e:
    raise Exception(f"❌ Request failed: {e}")